# Example: Advanced constraints in static inverse free-boundary equilibrium calculations

---

In this example notebook, we demonstrate some of the more advanced types of constraints and techniques that can be employed in the inverse solver. 

#### Instatiate the objects

As before, we start by instantiating the machine, equilibrium, profiles, and solver objects. 

In [ ]:
# build machine
from freegsnke import build_machine
tokamak = build_machine.tokamak(
    active_coils_path="../machine_configs/MAST-U/MAST-U_like_active_coils.pickle",
    passive_coils_path="../machine_configs/MAST-U/MAST-U_like_passive_coils.pickle",
    limiter_path="../machine_configs/MAST-U/MAST-U_like_limiter.pickle",
    wall_path="../machine_configs/MAST-U/MAST-U_like_wall.pickle",
)

from freegsnke import equilibrium_update
eq = equilibrium_update.Equilibrium(
    tokamak=tokamak,      # provide tokamak object
    Rmin=0.1, Rmax=2.0,   # radial range
    Zmin=-2.2, Zmax=2.2,  # vertical range
    nx=65,                # number of grid points in the radial direction (needs to be of the form (2**n + 1) with n being an integer)
    ny=129,               # number of grid points in the vertical direction (needs to be of the form (2**n + 1) with n being an integer)
    # psi=plasma_psi
)

# initialise the profiles
from freegsnke.jtor_update import ConstrainPaxisIp
profiles = ConstrainPaxisIp(
    eq=eq,        # equilibrium object
    paxis=8e3,    # profile object
    Ip=6e5,       # plasma current
    fvac=0.5,     # fvac = rB_{tor}
    alpha_m=1.8,  # profile function parameter
    alpha_n=1.2   # profile function parameter
)

from freegsnke import GSstaticsolver
GSStaticSolver = GSstaticsolver.NKGSsolver(eq)    

### Constraints

In addition to the `null points`, `isoflux set`, and `coil_current_limits` constraints introduced in the previous notebook, additional methods are available for more advanced control of the inverse problem.

We can also constrain:

- **Flux values (`psi_vals`)**  
  If the flux is known at a location $(R_j, Z_j)$, we can impose
  $$
  \psi(R_j, Z_j) = \psi_j^{\text{target}}.
  $$
  More generally, one could prescribe $\psi(R,Z)$ over a region (e.g. a full flux map). Flux values are typically difficult to estimate a priori to simulation and so the following constraint is often more useful. 

- **Normalised flux values (`psi_norm_limits`)**  
  At a location $(R_j, Z_j)$, we can impose upper (or lower) bound constraints on the normalised flux 
  $$
  \hat{\psi}(R,Z) = \frac{\psi(R,Z) - \psi_{axis}}{\psi_{boundary} - \psi_{axis}},
  $$ 
  such that 
  $$
  \hat{\psi}(R_j, Z_j) \leq \hat{\psi}_j^{\text{target}}.
  $$
  Note that $\psi_{axis}$ and $\psi_{boundary}$ are the values of the flux on the magnetic axis and plasma boundary, respectively.
  As we'll see, this can be useful for setting explicit constraints on flux behaviour near the wall if there are certain safety limits to adhere to.

Once again, we stress that users should be mindful of specifying too many constraints that may conflict with one another during an inverse solve. For example, a given normalised $\psi$ constraint may be impossible/difficult to achieve given other isoflux constraints or coil current limits. More generally, specifying too many constraints could make the problem ill-posed and reduce the overall quality of the final solution. 

Recall that if you find that the solver is violating the limit (inequality) constraints (coil limits and normalised $\psi$), you should look to reduce the number of constraints or try modifying the `mu_*` parameters. These parameters control how much a violation of the constraint is penalised in the solver, and increasing this penalty (by increasing `mu_*`) will increase the likelihood the constraint is adequately satisfied. Again, it may also be that the constraint is impossible to satisfy given your other constraints.

In the following, we will show how to use the normalised flux constraints to control the normalised flux on the divertor nose.

In [ ]:
# as before, let's fix the Solenoid current
eq.tokamak.set_coil_current('Solenoid', 5000)
eq.tokamak['Solenoid'].control = False  # ensures the current in the Solenoid is fixed

In [ ]:
import numpy as np 
from freegsnke.inverse import Inverse_optimizer

Rx = 0.6      # X-point radius
Zx = 1.1      # X-point height
Rout = 1.4    # outboard midplane radius
Rin = 0.34    # inboard midplane radius

# set desired null_points locations (this can include X-point and O-point locations)
null_points = [[Rx, Rx], [Zx, -Zx]]

# set desired isoflux constraints with format 
# isoflux_set = [isoflux_0, isoflux_1 ... ] 
# with each isoflux_i = [R_coords, Z_coords]
isoflux_set = np.array([
    [[Rx, Rx, Rin, Rout], [Zx, -Zx, 0.,0.]]
    ])

# set the coil current limits (upper and lower)
# coil ordering in this case: PX,  D1,  D2,  D3,  Dp,  D5,  D6,  D7,  P4,  P5,  P6
coil_current_limits = [
    [5e3, 9e3, 9e3, 7e3, 7e3, 5e3, 4e3, 5e3, 0.0, 0.0, None],
    [-5e3, -9e3, -9e3, -7e3, -7e3, -5e3, -4e3, -5e3, -10e3, -10e3, None]
]

# normalised psi constraints set with format:
# [R, Z, psiN, 1] --> ψ̂(R,Z) ≥ psiN  (the +1 or -1 defines ≥ or ≤)
# [R, Z, psiN, -1] --> ψ̂(R,Z) ≤ psiN
psi_norm_limits = [
    [0.82, -1.55, 1.2, 1],
    [0.82, -1.55, 1.3, -1],
]

# instantiate the freegsnke constrain object
constrain = Inverse_optimizer(
    null_points=null_points,
    isoflux_set=isoflux_set,
    coil_current_limits=coil_current_limits,
    psi_norm_limits=psi_norm_limits,
)

# if you find coil limits are being violated or an undesireable solution is being produced,
# you can try increasing the penalty factor for violating the coil limits
# (here we just set it to its default value of 1e5)
constrain.mu_coils = 1e5

# Similarly, you can define how much normalised psi constraints are penalised
# (the default value is 1e6)
constrain.mu_psi_norm = 1e7

We solve, as in the previous example, by passing the equilibrium, profiles, and constraints to the static solver. Again, we apply regularisation to encourage a solution with low coil currents.

In [ ]:
GSStaticSolver.solve(eq=eq, 
                     profiles=profiles, 
                     constrain=constrain, 
                     target_relative_tolerance=1e-6,
                     target_relative_psit_update=1e-3,
                     verbose=True, # print output
                     l2_reg=np.array([1e-12]*10+[1e-6]), 
                     )

In [ ]:
import matplotlib.pyplot as plt

# plot the resulting equilbria 
fig1, ax1 = plt.subplots(1, 1, figsize=(4, 8), dpi=80)
ax1.grid(True, which='both')
eq.plot(axis=ax1, show=False)
eq.tokamak.plot(axis=ax1, show=False)
constrain.plot(axis=ax1,show=True)
ax1.set_xlim(0.1, 2.15)
ax1.set_ylim(-2.25, 2.25)
plt.tight_layout()

Finally, you can check that the normalised $\psi$ constraint has indeed been satisfied.

In [ ]:
for psi_con in psi_norm_limits:
    sign = ">=" if psi_con[3] >= 0 else "<="
    print(f"Psi norm at ({psi_con[0]}, {psi_con[1]}) = {eq.psiNRZ(psi_con[0], psi_con[1]):.3f} ({sign} {psi_con[2]})")